In [1]:
import cv2
import pyaudio
import socket
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def setup_server():
    try:
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind(('172.20.10.6', 8000))
        server_socket.listen(1)
        logging.info("Waiting for a connection...")
        client_socket, addr = server_socket.accept()
        logging.info(f"Connected to {addr}")
        return server_socket, client_socket
    except Exception as e:
        logging.error(f"Error setting up server: {e}")
        raise

def receive_control_message(client_socket):
    try:
        control_message = client_socket.recv(1024).decode()
        logging.debug(f"Received control message: {control_message}")
        enable_camera, enable_audio = control_message.split(',')
        enable_camera = enable_camera.lower() == 'true'
        enable_audio = enable_audio.lower() == 'true'
        return enable_camera, enable_audio
    except Exception as e:
        logging.error(f"Error receiving control message: {e}")
        raise

def send_data(client_socket, enable_camera, enable_audio, cap, stream):
    try:
        while True:
            if enable_camera:
                ret, frame = cap.read()
                if not ret:
                    logging.error("Error capturing video frame")
                    continue
                encoded_frame = cv2.imencode('.jpg', frame)[1].tobytes()
                logging.debug(f"Encoded video frame length: {len(encoded_frame)}")
            else:
                encoded_frame = b''

            if enable_audio:
                audio_data = stream.read(1024)
                logging.debug(f"Audio data length: {len(audio_data)}")
            else:
                audio_data = b''

            data_to_send = len(encoded_frame).to_bytes(4, byteorder='big') + encoded_frame + audio_data
            try:
                client_socket.sendall(data_to_send)
            except BrokenPipeError:
                logging.error("Broken pipe encountered. Client disconnected.")
                break

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except Exception as e:
        logging.error(f"Error sending data: {e}")
        raise
    finally:
        cap.release()
        stream.stop_stream()
        stream.close()
        client_socket.close()

def main():
    cap = cv2.VideoCapture(0)
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)

    try:
        server_socket, client_socket = setup_server()
        enable_camera, enable_audio = receive_control_message(client_socket)
        send_data(client_socket, enable_camera, enable_audio, cap, stream)
    except KeyboardInterrupt:
        logging.info("Keyboard interrupt detected. Exiting...")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
    finally:
        server_socket.close()

if __name__ == "__main__":
    main()


2024-05-11 02:25:47,492 - INFO - Waiting for a connection...
2024-05-11 02:25:51,356 - INFO - Connected to ('172.20.10.6', 53948)
2024-05-11 02:25:51,357 - DEBUG - Received control message: True,False
2024-05-11 02:25:51,372 - DEBUG - Encoded video frame length: 195933
2024-05-11 02:25:51,423 - DEBUG - Encoded video frame length: 196810
2024-05-11 02:25:51,477 - DEBUG - Encoded video frame length: 202041
2024-05-11 02:25:51,508 - DEBUG - Encoded video frame length: 204153
2024-05-11 02:25:51,538 - DEBUG - Encoded video frame length: 204195
2024-05-11 02:25:51,572 - DEBUG - Encoded video frame length: 204055
2024-05-11 02:25:51,608 - DEBUG - Encoded video frame length: 205644
2024-05-11 02:25:51,608 - ERROR - Error sending data: [Errno 32] Broken pipe
2024-05-11 02:25:51,850 - ERROR - An unexpected error occurred: [Errno 32] Broken pipe
